In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# CSV 파일에서 titleId 값을 가져오기
title_df = pd.read_csv("kakao_listly.csv")  # kakao_listly.csv 파일을 불러오기
id_list = title_df['url'].astype(str).tolist()  # url 칼럼의 값들을 문자열로 변환하여 리스트로 저장
print(len(id_list))
print(id_list[:1])

# WebDriver 설정
options = Options()
options.add_argument('headless')  # 헤드리스 모드
options.add_argument('window-size=1920x1080')  # 윈도우 크기 설정
options.add_argument("disable-gpu")  # GPU 비활성화
driver = webdriver.Chrome(options=options)

# 데이터 저장 리스트
data = []

# URL 순회하며 정보 수집
for url in id_list[:1]:  # 예시로 1개만 크롤링
    # 오버뷰 페이지 URL 생성 (제목 추출)
    overview_url = f"{url}?tab_type=overview"
    driver.get(overview_url)
    time.sleep(2)  # 페이지가 완전히 로드될 시간을 줍니다.

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제목 추출 (오버뷰 페이지에서)
    title_span = soup.find("span", class_="font-large3-bold mb-3pxr text-ellipsis break-all text-el-70 line-clamp-2")
    title_text = title_span.get_text(strip=True) if title_span else "제목을 찾을 수 없음"

    # 어바웃 페이지 URL 생성
    about_url = f"{url}?tab_type=about"
    driver.get(about_url)
    time.sleep(2)  # 페이지가 완전히 로드될 시간을 줍니다.

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 줄거리
    story_span = soup.find("span", class_="font-small1 mb-8pxr block whitespace-pre-wrap break-words text-el-70")
    story_text = story_span.get_text(strip=True) if story_span else "줄거리를 찾을 수 없음"

    # 해시태그
    hashtags = []
    hashtag_spans = soup.find_all("span", class_="font-small2-bold text-ellipsis text-el-70 line-clamp-1")
    for hashtag in hashtag_spans:
        hashtags.append(hashtag.get_text(strip=True))
    hashtags_text = ', '.join(hashtags) if hashtags else "해시태그를 찾을 수 없음"

    # 연령 등급
    # 연령 등급은 `span` 태그의 텍스트가 "전체이용가" 등으로 정확히 일치하는 텍스트입니다.
    age_rating_text = "연령 등급을 찾을 수 없음"
    info_divs = soup.find_all("div", class_="font-small1 flex w-full pt-6pxr")
    for div in info_divs:
        label_span = div.find("span")
        if label_span and "연령등급" in label_span.get_text():
            value_span = div.find_all("span")[-1]
            age_rating_text = value_span.get_text(strip=True)
            break

    
    # 저장
    data.append({
        'Title': title_text,
        'Story': story_text,
        'Hashtags': hashtags_text,
        'Age Rating': age_rating_text
    })

    # 10개 확인용
    print(f"현재 작품: {title_text}")
    time.sleep(3)  # 데이터 확인을 위한 시간

# 드라이버 종료
driver.quit()

# DataFrame 생성 및 출력
df = pd.DataFrame(data)[['Title', 'Story', 'Hashtags', 'Age Rating']]

# CSV로 저장
df.to_csv('kakao_crawling_about.csv', index=False, encoding='utf-8-sig')


2677
['https://page.kakao.com/content/50866481']
현재 작품: 나 혼자만 레벨업
       Title                                              Story  \
0  나 혼자만 레벨업  10여 년 전, \n다른 차원과 이쪽 세계를 이어 주는 통로 ‘게이트’가 열리고 \...   

                                      Hashtags Age Rating  
0  #생존, #먼치킨, #시스템, #천재, #노력, #성장, #이능력, #플레이어      전체이용가  
